# Importando bibliotecas

In [1]:
#pyarrow para manipulação de grandes conjuntos de dados
import pyarrow.parquet as pq
import pyarrow as pa

# biblotecas para manipulação de dados
import pandas as pd
import datatable as dt
import numpy as np

#Bibliotecas auxiliares
import os
from functools import reduce
import re
import pickle
import gc
import tqdm

#Biblioteca propria
import sys
sys.path.append("../src/")
from eda.eda import describe
from io_pyarrow.io_pyarrow import pyarrow_read_csv,write_table_from_pandas, read_table_to_pandas #leitura e escrita de arquivos csv grandes

%reload_ext watermark
%watermark --iversions

datatable: 0.11.1
numpy    : 1.20.1
pyarrow  : 4.0.0
tqdm     : 4.60.0
sys      : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
re       : 2.2.1
pandas   : 1.2.2



___

# Importando arquivo

In [2]:
#Definindo path dos arquivos
path_data = "../data/raw/"

## Dados da FAPESP
<b> Descrição </b>: Pasta de arquivos, com dados anonimizados de pacientes que fizeram teste para COVID-19 a partir de 1/11/2019 e respectivos resultados de exames laboratoriais, contendo dentre outros o identificador anonimizado do paciente  <br>
fonte: https://repositoriodatasharingfapesp.uspdigital.usp.br/

### Hospital Sírio Libanes

In [4]:
%%time
hsl_exames = pyarrow_read_csv(path_data + "FAPESP/HSL_Exames_3.csv",sep="|")
print(hsl_exames.shape)
hsl_pacientes = pyarrow_read_csv(path_data + "FAPESP/HSL_Pacientes_3.csv",sep="|")
print(hsl_pacientes.shape)
hsl_desfechos = pyarrow_read_csv(path_data + "FAPESP/HSL_Desfechos_3.csv",sep="|")
print(hsl_desfechos.shape)

(1463834, 9)
(8971, 7)
(42691, 8)
Wall time: 1.55 s


___

# Analisando a bases de dados

### Ajustando dados do Fleury

In [5]:
hsl = hsl_exames.merge(hsl_desfechos,
                       on = ["ID_PACIENTE","ID_ATENDIMENTO"],
                       how = 'left').merge(hsl_pacientes,
                                           on = "ID_PACIENTE",
                                           how = 'left')

describe(hsl)

Quantidade de linhas: 1463834


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,ID_PACIENTE,object,0,0.0%,8971,-,-,-,-,-,-,-,-,-,-
1,ID_ATENDIMENTO,object,0,0.0%,19550,-,-,-,-,-,-,-,-,-,-
2,DT_COLETA,object,0,0.0%,308,-,-,-,-,-,-,-,-,-,-
3,DE_ORIGEM,object,0,0.0%,95,-,-,-,-,-,-,-,-,-,-
4,DE_EXAME,object,0,0.0%,665,-,-,-,-,-,-,-,-,-,-
5,DE_ANALITO,object,0,0.0%,954,-,-,-,-,-,-,-,-,-,-
6,DE_RESULTADO,object,0,0.0%,15520,-,-,-,-,-,-,-,-,-,-
7,CD_UNIDADE,object,0,0.0%,54,-,-,-,-,-,-,-,-,-,-
8,DE_VALOR_REFERENCIA,object,0,0.0%,670,-,-,-,-,-,-,-,-,-,-
9,DT_ATENDIMENTO,object,128224,8.8%,308,-,-,-,-,-,-,-,-,-,-


In [6]:
hsl_ = hsl.copy()

#### DT_COLETA

In [7]:
#DT_COLETA é um dado no formato de data que deve ser convertido
hsl_.DT_COLETA = pd.to_datetime(hsl_.DT_COLETA,errors='coerce')
print("Quantidade de registros que não puderam ser convertidos:",hsl_.DT_COLETA.isna().sum())

Quantidade de registros que não puderam ser convertidos: 0


#### DE_ORIGEM

<font color = 'red'><b> Verificar se existe alguma forma de reduzir esse volume de DE_ORIGEM</b></font>

De acordo com o dicionario de dados, o formato deveria ser composto por elementos de 4 caracteres, mas não é isso que ocorre aqui

In [23]:
hsl_.query("'Unidades de Internação' == DE_ORIGEM").groupby(["DE_ORIGEM","DE_CLINICA"]).ID_PACIENTE.count()

DE_ORIGEM               DE_CLINICA                
Unidades de Internação  Ambulatório                      233
                        Ambulância                      2658
                        CL Médica Síndromes Virais       159
                        Cirurgia                       19930
                        Clínica Médica                325089
                        Consulta                         210
                        Cuidando de Quem Cuida            26
                        Exames                           929
                        Hospital Dia                     101
                        Oncologia                        125
                        Ortopedia                       1697
                        Outros                          4194
                        Pediatria                         87
                        Peq. Cirurgias                    46
                        Procedimentos                     16
                        Procedimen

In [17]:
hsl_.DE_ORIGEM.value_counts().to_dict()

{'UTI': 718522,
 'Unidades de Internação': 363883,
 'Pronto Socorro': 146810,
 'Recepção do Centro Diagnóstico': 120367,
 'Centro de Oncologia': 16483,
 'Laboratório de Patologia Clínica': 13403,
 'Recepção CDI - 2º andar - Itaim': 12764,
 'Hemodiálise': 10768,
 'Programa Cuidando de Quem Cuida': 9468,
 'Oncologia Clínica - Brasília': 8312,
 'Apoio Administrativo - Jardins': 4432,
 'Tomografia': 3465,
 'Ultrassonografia': 3240,
 'Ultrassonografia - Jardins': 2038,
 'Centro Cirurgico': 1732,
 'Laboratório de Patologia Clínica - Brasília IV': 1697,
 'Oncologia Clínica - Itaim': 1669,
 'Ecocardiografia': 1645,
 'Laboratório de Anatomia Patológica': 1530,
 'Oncologia Clínica - Brasília II': 1422,
 'Laboratório de Patologia Clínica - Itaim': 1283,
 'Tomografia - Itaim': 1275,
 'Radiologia Geral': 1223,
 'Check Up - Itaim': 1214,
 'Central de Agendamento do SADT': 1199,
 'Ultrassonografia - Itaim': 1194,
 'Medicina do Trabalho': 849,
 'Ressonância Magnética': 825,
 'Laboratório de Patologia 

#### CD_UNIDADE

In [28]:
#Existem alguns casos vazios para CD_Unidade
hsl_.CD_UNIDADE.value_counts()

                       359175
/mm3                   224443
mg/dL                  202485
%                      124745
mL/minuto               95140
g/dL                    67158
mEq/L                   64908
U/L                     49367
mmHg                    41930
mmol/L                  39115
segundos                34949
fL                      31175
pg                      29541
milhões/mm3             29541
ng/mL                   26291
g/L                      7078
mL/min/1,73 m2           4636
ng/dL                    4515
pg/mL                    4158
microg/L                 3661
microg/dL                2953
UI/mL                    2031
microU/mL                1951
UA/mL                    1904
U/mL                     1490
UI/L                     1319
ng/L                     1192
mm                       1090
mUI/L                    1002
mU/L                      913
micromol/L                846
microgramas/mL            789
mg/L                      588
pmol/L    

In [29]:
#Existem algumas unidades vazias, portanto vamos aplicar NaN a elas e transformar em categoria
hsl_.CD_UNIDADE = hsl_.CD_UNIDADE.replace({"":np.nan})
hsl_.CD_UNIDADE = pd.Categorical(hsl_.CD_UNIDADE,categories=hsl_.CD_UNIDADE.value_counts().index)

#### DE_VALOR_REFERENCIA

In [40]:
#Existem alguns valores vazios em valor de referencia
hsl_.DE_VALOR_REFERENCIA.value_counts()

                             279462
Superior a 60                 99776
Ver resultado tradicional     38411
1.700 a 7.000                 34000
0,70 a 1,30                   32694
                              ...  
13,9 a 85,5                       1
2161 a 5691                       1
33 a 258                          1
inferior a 1/1600                 1
5000 a 12000                      1
Name: DE_VALOR_REFERENCIA, Length: 670, dtype: int64

In [41]:
# Vamos pegar esses valores em branco e transformá-los
hsl_.DE_VALOR_REFERENCIA = hsl_.DE_VALOR_REFERENCIA.replace({"":np.nan})

#### DT_ATENDIMENTO

In [46]:
hsl_.DT_ATENDIMENTO = pd.to_datetime(hsl_.DT_ATENDIMENTO,errors = 'coerce')
print("Quantidade de registros que não puderam ser convertidos:",hsl_.DT_ATENDIMENTO.isna().sum())

Quantidade de registros que não puderam ser convertidos: 128224


In [49]:
#Alguns registros não puderam ser convertidos
#Motivo: os valores já era registros faltantes antes da conversão
pd.DataFrame({"dt_convertida": hsl_.DT_ATENDIMENTO , "dt_original": hsl.DT_ATENDIMENTO}).query("dt_convertida != dt_convertida").dt_original.value_counts(dropna=False)

NaN    128224
Name: dt_original, dtype: int64

#### DE_TIPO_ATENDIMENTO

In [62]:
#São apenas 5 possíveis, então, vamos converter para dados categoricos
hsl_.DE_TIPO_ATENDIMENTO.value_counts(dropna = False)

Internado             955519
Externo               187027
Pronto Atendimento    147631
NaN                   128224
Ambulatorial           45433
Name: DE_TIPO_ATENDIMENTO, dtype: int64

In [63]:
hsl_.DE_TIPO_ATENDIMENTO = pd.Categorical(hsl_.DE_TIPO_ATENDIMENTO,categories=hsl_.DE_TIPO_ATENDIMENTO.value_counts().index)

#### DE_CLINICA

In [67]:
hsl_.DE_CLINICA.value_counts(dropna=False)

Clínica Médica                924492
Procedimentos                 172239
NaN                           128224
CL Médica Síndromes Virais     76404
Cirurgia                       53320
Oncologia                      22076
Outros                         19991
Ambulância                     17200
Exames                         15529
Hemodiálise                    10424
Cuidando de Quem Cuida         10148
Ortopedia                       4252
Pediatria                       2948
Geriatria                       1374
Check-up                         954
Medicina Trabalho                857
Neurologia                       785
Fast track                       680
Cardiologia                      590
Ambulatório                      413
Consulta                         362
Procedimentos Enfermagem         167
Hospital Dia                     101
Centro Reabilitação               67
Centro de Infusão                 63
Itaim - Clínica Médica            52
Retorno Digital Adulto            47
P

In [68]:
#Convertendo para dados numericos
hsl_.DE_CLINICA = pd.Categorical(hsl_.DE_CLINICA,categories=hsl_.DE_TIPO_ATENDIMENTO.value_counts().index)

#### IC_SEXO

#### AA_NASCIMENTO

#### CD_PAIS

#### CD_UF

#### CD_MUNICIPIO

#### CD_CEPREDUZIDO

#### DE_RESULTADO

De acordo com o dicionário de dados:

<i>"Se DE_ANALITO exige valor numérico, NNNN se inteiro ou NNNN,NNN se casas decimais
Se DE_ANALITO exige qualitativo, String com domínio restrito 
Se DE_ANALITO por observação microscópica, String conteúdo livre "</i>

Portanto, vamos fazer essa divisão

In [30]:
hsl_.DE_RESULTADO.value_counts()

superior a 60                                                                                                       39933
superior a 90                                                                                                       23177
normais                                                                                                             15655
não foram observados caracteres tóxico-degenerativos nos neutrófilos, não foram observadas atipias linfocitárias    15628
NEGATIVO                                                                                                            13140
                                                                                                                    ...  
424,7                                                                                                                   1
49,87                                                                                                                   1
1747                    

In [31]:
def func_filtro_numerico_DE_RESULTADO(x):
    if((re.search("[0-9]+",x.lower()) == None) & (re.search("[0-9]+,[0-9]+",x.lower()) == None)):
        return False
    elif(re.search("[0-9]+,[0-9]+",x.lower()) != None):
        if(re.search("[0-9]+,[0-9]+",x.lower()).span()[1] - re.search("[0-9]+,[0-9]+",x.lower()).span()[0] == len(x)):
            return True
        else:
            return False
    else:
        if(re.search("[0-9]+",x.lower()).span()[1] - re.search("[0-9]+",x.lower()).span()[0] == len(x)):
            return True
        else:
            return False

In [34]:
filtro_numerico_DE_RESULTADO = hsl_.DE_RESULTADO.apply(func_filtro_numerico_DE_RESULTADO)

In [35]:
hsl_["DE_RESULTADO_NUMERICO"] = pd.to_numeric(hsl_.DE_RESULTADO[filtro_numerico_DE_RESULTADO].str.replace(",","."),errors="coerce")
hsl_["DE_RESULTADO_NAO_NUMERICO"] = hsl_.DE_RESULTADO[~filtro_numerico_DE_RESULTADO]

#### DE_ANALITO

In [27]:
hsl_.DE_ANALITO.value_counts()

Creatinina                                         51321
Hemoglobina                                        31921
Hematócrito                                        30120
Plaquetas                                          29899
Leucócitos                                         29543
                                                   ...  
Cultura para Bacilo gram - Parcial                     1
Rast - F232 - nGal d 2 Ovoalbumina                     1
Bk (Baar), Cultura (Lavado Bronquico) - Parcial        1
Receptor de Fosfolipase A2 (PLA-2R)                    1
Normetanefrina, urina                                  1
Name: DE_ANALITO, Length: 954, dtype: int64

#### DE_EXAME

In [25]:
hsl_.DE_EXAME.value_counts()

Hemograma                                                     644292
Calculo da Estimativa da Taxa, de Filtracao Glomular, soro    124720
Gasometria Venosa                                              68844
Gasometria Arterial                                            56946
Urina Tipo I - Jato Medio                                      44139
                                                               ...  
Fator de crescimento do endotélio vascular (VEGF), plasma          1
Cevada, Soro                                                       1
Beta Caroteno (Pro Vit. A)                                         1
Insulina, Livre, Soro                                              1
Prova do Laco                                                      1
Name: DE_EXAME, Length: 665, dtype: int64